In [1]:
import requests
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
from bs4 import BeautifulSoup as bs
import lxml
import time

In [2]:
page_link = 'https://knowyourmeme.com/memes/all/page/1'
headers = {
    'User-Agent': UserAgent().firefox
}

In [3]:
response = requests.get(page_link, headers=headers)
response

<Response [200]>

In [4]:
html = response.content
soup = bs(html, 'lxml')

In [5]:
soup.html.head.title

<title>All Entries | Know Your Meme</title>

In [6]:
soup.html.head.title.text

'All Entries | Know Your Meme'

In [7]:
obj = soup.find('a', attrs={'class':'photo'})
obj

<a class="photo left" href="/editorials/interviews/me-gustas-creator-reveals-the-untold-origin-story-of-the-iconic-rage-face-and-how-her-identity-was-stolen-by-an-nft-scammer-trying-to-profit-off-her-meme" target="_self"><img alt="Me_gusta_may_oswald_interview_banner" data-src="https://i.kym-cdn.com/featured_items/icons/wide/000/012/771/Me_Gusta_May_Oswald_Interview_Banner.jpg" height="112" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Me Gusta’s Creator Reveals The Untold Origin Story Of The Iconic Rage Face" width="198"/> <div class="info abs"> <div class="c"> Me Gusta’s Creator Reveals The Untold Origin Story Of The Iconic Rage Face </div> </div> </a>

In [8]:
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
obj

<a class="photo" href="/memes/captain-delilah"><img alt="Captain Delilah" data-src="https://i.kym-cdn.com/entries/icons/medium/000/037/196/Screenshot_20210504-215719_Adobe_Photoshop_Mix.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Captain Delilah"/> <div class="entry-labels"> <span class="label label-deadpool"> Deadpool </span> </div> </a>

In [9]:
print('Тип данных до вытаскивания ссылки', type(obj))
print('Тип данных после вытаскивания ссылки', type(obj.attrs['href']))

Тип данных до вытаскивания ссылки <class 'bs4.element.Tag'>
Тип данных после вытаскивания ссылки <class 'str'>


In [10]:
meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'] )
meme_links[:3]

[<a class="photo" href="/memes/captain-delilah"><img alt="Captain Delilah" data-src="https://i.kym-cdn.com/entries/icons/medium/000/037/196/Screenshot_20210504-215719_Adobe_Photoshop_Mix.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Captain Delilah"/> <div class="entry-labels"> <span class="label label-deadpool"> Deadpool </span> </div> </a>,
 <a class="photo" href="/memes/triggered-stan"><img alt="Triggered Stan" data-src="https://i.kym-cdn.com/entries/icons/medium/000/037/195/Stan_No_It_Not_Cool.png" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Triggered Stan"/> <div class="entry-labels"> <span class="label label-deadpool"> Deadpool </span> </div> </a>,
 <a class="photo" href="/memes/don-cheadle-word-of-the-day"><img alt="Don Cheadle Word of the Day" data-src="https://i.kym-cdn.com/entries/icons/medium/000/037/194/cover4.jpg" src="https://s.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif

Осталось очистить полученный список от мусора

In [11]:
meme_links = [link.attrs['href'] for link in meme_links]

In [12]:
meme_links[:10]

['/memes/captain-delilah',
 '/memes/triggered-stan',
 '/memes/don-cheadle-word-of-the-day',
 '/memes/please-give-me-a-whole-tray-of-peas',
 '/memes/astronaut-in-the-ocean',
 '/memes/jonkers-is-now-working-on-a-special-liquid',
 '/memes/there-are-federal-agents-outside-my-house',
 '/memes/im-sick-of-crying-but-inside-im-dying',
 '/memes/events/billie-eilish-vogue-cover',
 '/memes/grabovoi-cheat-codes']

In [13]:
len(meme_links)

16

In [14]:
def get_page_links(page_number):
    """
    Возвращает список ссылок на мемы, полученный с текущей страницы
    
    page_number: int/string
        номер страницы для парсинга
        
    """
    # Составляем ссылку на страницу поиска
    page_link = f'https://knowyourmeme.com/memes/all/page/{page_number}'
    
    # Запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().firefox})
    
    if not response.ok:
        # Если версер нам отказал - вернем пустой лист для текущей страницы
        return []
    
    html = response.content
    soup = bs(html, 'lxml')

    # наконец, ищем ссылки на мемы и очищаем их от ненужных тегов
    
    meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
    meme_links = ['https://knowyourmeme.com' + link.attrs['href'] for link in meme_links]
    
    return meme_links

Протестируем функцию и убедимся, что все хорошо

In [15]:
meme_links = get_page_links(1)
meme_links[:2]

['https://knowyourmeme.com/memes/captain-delilah',
 'https://knowyourmeme.com/memes/triggered-stan']

In [16]:
meme_links = get_page_links(2)
meme_links[:2]

['https://knowyourmeme.com/memes/bracket-explanations',
 'https://knowyourmeme.com/memes/sites/dark-simpsons']

In [17]:
meme_page = 'https://knowyourmeme.com/memes/doge'
response = requests.get(meme_page, headers={'User-Agent': UserAgent().firefox})

html = response.content
soup = bs(html, 'lxml')

Посмотрим, как можно вытащить статистику комментариев, а так же количество загруженных видео и фото

In [18]:
views = soup.find('div', attrs={'class':'views'})
views

<div class="views" title="13,651,482 Views"></div>

In [19]:
views.attrs

{'class': ['views'], 'title': '13,651,482 Views'}

In [20]:
views = int((views['title'].split()[0]).replace(',', ''))
views

13651482

**Снова запихнем всё это в небошьшую функцию**

In [21]:
def get_stats(soup, stats):
    """
    Возвращает очищенное число просмотров/комментариев...
    
    soup: объект bs4.BeautifulSoup
        суп текущей страницы
        
    stats: string
        views/videos/photos/comments
    
    """
    try:
        obj = soup.find('div', attrs={'class': stats})
#         obj = obj.find('a').text
        obj = int((obj['title'].split()[0]).replace(',', ''))
        
    except:
        obj = None
    return obj

In [22]:
views = get_stats(soup, stats='views')
videos = get_stats(soup, stats='videos')
photos = get_stats(soup, stats='photos')
comments = get_stats(soup, stats='comments')

print("Просмотры: {}\nВидео: {}\nФото: {}\nКомментарии: {}".format(views, videos, photos, comments))

Просмотры: 13651482
Видео: 75
Фото: 1766
Комментарии: 920


In [23]:
meme_page = 'https://knowyourmeme.com/memes/doge'
response = requests.get(meme_page, headers={'User-Agent': UserAgent().firefox})

html = response.content
soup = bs(html, 'lxml')

In [24]:
date = soup.find('abbr', attrs={'class': 'timeago'}).attrs['title']
date

'2013-07-24T16:29:55-04:00'

In [25]:
properties = soup.find('div', attrs={'class':'tooltip-popup'})
meme_status = properties.find('span')
meme_status = 'empty' if not meme_status else meme_status.text.strip()
meme_status

'confirmed'

In [26]:
def get_properties(soup):
    """
    Возвращает (tuple) с названиемб статусом, типомб
    годом, местом происхождения и тегами
    
    soup: объект bs4.BeautifulSoup
        суп текущей страницы
    """
    
    # Название - идет с самы большим заголовком h1, легко найти
    meme_name = soup.find('section', attrs={'class': 'info wide'}).find('h1').text.strip()
    
    # Достаём все данные справа от картинки
    properties = soup.find('div', attrs={'class': 'tooltip-popup'})
    
    # Статус идёт первым, можно не уточнять класс
    meme_status = properties.find('span')
    # Oneliner, заменяющий try-except: если тэга нет в properties, вернётся объект NoneType,
    # у которого аттрибут text отсутствует, и в этом случае он заменится на пустую строку
    meme_status = 'empty' if not meme_status else meme_status.text.strip()
    
    # Тип мема - обладает уникальным классом
    meme_type = properties.find('dev', attrs={'class': 'details'})
    print(meme_type)
    meme_type = 'empty' if not meme_type else meme_type.text
    
    # Год происхождения первоистояника можно найти после заголовка Year
    meme_origin_year = properties.find(text='\nYear\n')
    meme_origin_year = '' if not meme_origin_year else meme_origin_year.parent.find_next()
    meme_origin_year = meme_origin_year.strip()
    
    # Сам первоисточник
    meme_origin_place = properties.find('div', attrs={'class': 'entry-section'})
    meme_origin_place = '' if not meme_origin_place else meme_origin_place.text.strip()
    
    # Теги связанные с мемом
    #meme_tags = properties.find('div', attrs={'class': 'tags'}).find('a')
    #meme_tags = '' if not meme_tags else meme_tags.text.strip()
    
    return meme_name, meme_status, meme_type, meme_origin_year, meme_origin_place #, meme_tags

In [27]:
get_properties(soup)

None


('Doge', 'confirmed', 'empty', '', '')

In [28]:
def check_ip():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = bs(ip, 'lxml')
    print(soup.find('body').text)

In [29]:
check_ip()

Current IP Address: 176.106.242.94


In [30]:
import socks
import socket

socks.set_default_proxy(socks.SOCKS5, 'localhost', 9150)
socket.socket = socks.socksocket

In [31]:
check_ip()

ConnectionError: HTTPConnectionPool(host='checkip.dyndns.org', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EBF35ED0A0>: Failed to establish a new connection: Error connecting to SOCKS5 proxy localhost:9150: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))

In [ ]:
for i in range(10):
    check_ip()
    time.sleep(5)

In [34]:
from tqdm import trange, tqdm
import time

for i in tqdm(range(10)):
    i *= i + 1
    time.sleep(1)
    print(i)

 10%|████████▎                                                                          | 1/10 [00:01<00:09,  1.00s/it]

0


 20%|████████████████▌                                                                  | 2/10 [00:02<00:08,  1.00s/it]

2


 30%|████████████████████████▉                                                          | 3/10 [00:03<00:07,  1.01s/it]

...

56


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:09<00:01,  1.00s/it]

72


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.00s/it]

90
